### Исследуем таблицы — выведем первые строки

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#устанавливаем параметры

db_config = {'user': 'praktikum_student', #имя пользователя
'pwd': 'Sdf4$2;d-d30pp', #пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, #порт подключения
'db': 'data-analyst-final-project-db'} #название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
query = '''SELECT *
           FROM books'''

In [4]:
books = pd.io.sql.read_sql(query, con = engine) 

In [5]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
query = '''SELECT *
           FROM authors'''

In [7]:
authors = pd.io.sql.read_sql(query, con = engine) 

In [8]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
query = '''SELECT *
           FROM publishers'''

In [10]:
publishers = pd.io.sql.read_sql(query, con = engine)

In [11]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [12]:
query = '''SELECT *
           FROM ratings'''

In [13]:
ratings = pd.io.sql.read_sql(query, con = engine)

In [14]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [15]:
query = '''SELECT *
           FROM reviews'''

In [16]:
reviews = pd.io.sql.read_sql(query, con = engine)

In [17]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Сделаем по одному SQL-запросу для решения каждого задания

#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [18]:
query = '''SELECT COUNT(book_id)
           FROM books
           WHERE CAST(publication_date AS timestamp) >  '2000-01-01' '''

In [19]:
data = pd.io.sql.read_sql(query, con = engine)

In [20]:
data

,count
0,819


После 1 января 2000 года вышло 819 книг. 

#### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [36]:
query = '''SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT r.review_id),
                  AVG(rating)
           FROM books AS b
           LEFT JOIN reviews AS r ON b.book_id=r.book_id
           LEFT JOIN ratings AS rt ON b.book_id=rt.book_id
           GROUP BY 1
           ORDER BY 3 DESC'''

In [37]:
data_1 = pd.io.sql.read_sql(query, con = engine)

In [38]:
data_1

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Всего книг в количестве (1000). Больше всего обзоров на книгу Twilight (7).

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [24]:
query = '''SELECT p.publisher,
                  COUNT(b.book_id)
           FROM publishers AS p
           JOIN books AS b ON p.publisher_id=b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 1'''

In [25]:
data_2 = pd.io.sql.read_sql(query, con = engine)

In [26]:
data_2

,publisher,count
0,Penguin Books,42


Penguin Books - издательство, которое выпустило наибольшее количество книг толще 50 страниц в количестве (42) 

#### Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками

In [52]:
query = '''SELECT data.author,                                
                  AVG(data.avg) 
           FROM (SELECT a.author AS author,
                        b.book_id,
                        AVG(rt.rating) AS avg,
                        COUNT(rt.rating)
                 FROM authors AS a
                 JOIN books AS b ON a.author_id = b.author_id
                 JOIN ratings AS rt ON b.book_id = rt.book_id
                 GROUP BY 1, 2
                 HAVING COUNT(rt.rating) >= 50) AS data
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 1'''

In [53]:
data_3 = pd.io.sql.read_sql(query, con = engine)

In [54]:
data_3

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой оценкой книг (при учете только книги с 50 и более оценками) стала J.K.Rowling/Mary GrandPré оценка (4.28) 

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [64]:
query = '''SELECT COUNT(review_id) / COUNT(DISTINCT username) AS avg
           FROM reviews 
           WHERE username IN (SELECT data.username
                                      FROM (SELECT username,
                                                   COUNT(rating)   
                                            FROM ratings
                                            GROUP BY 1
                                            HAVING COUNT(rating) > 50) AS data)'''

In [65]:
data_4 = pd.io.sql.read_sql(query, con = engine)

In [66]:
data_4

,avg
0,24


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, равняется (24) 

## Вывод

Мы провели исследование, выполнили пять SQL-запросов по предоставленной базе данных. 

1. Посчитали, сколько книг вышло после 1 января 2000 года - 819 книг. 
2. Для каждой книги посчитали количество обзоров и среднюю оценку. Выявили что, всего книг в количестве (1000). Больше всего обзоров на книгу Twilight (7).
3. Определили издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключили из анализа брошюры. Это издательство Penguin Books, выпустило наибольшее количество книг толще 50 страниц в количестве (42)
4. Определили автора с самой высокой средней оценкой книг — учитывали только книги с 50 и более оценками. Им стала J.K. Rowling/Mary GrandPré оценка (4.28)
5. Посчитали среднее количество обзоров от пользователей, которые поставили больше 50 оценок. Выяснили что, среднее количество обзоров от пользователей, которые поставили больше 50 оценок, равняется (24). 